In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from collections import OrderedDict
import torch
torch.manual_seed(0)
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
from abcd.local.paths import output_path
from abcd.data.read_data import get_subjects_events_sf, add_event_vars, add_subject_vars, filter_events, filter_subjects
import abcd.data.VARS as VARS
from abcd.data.define_splits import SITES, save_restore_sex_fmri_splits
from abcd.data.divide_with_splits import divide_events_by_splits
from abcd.data.var_tailoring.normalization import normalize_var
from abcd.data.pytorch.get_dataset import PandasDataset
from abcd.local.paths import core_path, output_path
from abcd.data.VARS import STRUCT_FILES, STRUCT_FEATURES

from abcd.models.regression.LinearRegressor import LinearRegressor
from abcd.training.RegressorTrainer import RegressorTrainer

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess
1. Fetch subject and events dataframes
2. Change sex assigned at birth labels to 0 (male), 1 (female)
3. Filter out all non-baseline events
4. Define target column (sex assigned at birth), and features columns (functional connectivities and structural features)
5. Normalize features
6. train/val/test split

In [3]:
# Fetch subjects and events
subjects_df, events_df = get_subjects_events_sf()
# Change to 0 (male) and 1 (female)
events_df.loc[events_df["kbi_sex_assigned_at_birth"] == 1.0, "kbi_sex_assigned_at_birth"] = float(0)
events_df.loc[events_df["kbi_sex_assigned_at_birth"] == 2.0, "kbi_sex_assigned_at_birth"] = float(1)
# Leave only the baseline visits
events_df = events_df.loc[(events_df['eventname'] == 'baseline_year_1_arm_1')]
print("Leaving only baseline visits, we have {} events".format(len(events_df)))

Leaving only baseline visits, we have 9086 events


In [4]:
# Define target and features
target_col = 'kbi_sex_assigned_at_birth'
labels = ["Male", "Female"]
feature_cols = list(VARS.NAMED_CONNECTIONS.keys()) + list(VARS.STRUCT_FEATURES.keys())

# Print distribution of baseline class
for val in set(events_df['kbi_sex_assigned_at_birth']):
    print('{} visits with {} target'.format(len(events_df.loc[events_df["kbi_sex_assigned_at_birth"] == val]), labels[int(val)]))

4704 visits with Male target
4382 visits with Female target


In [5]:
# Normalize features
for var_id in feature_cols:
    events_df = normalize_var(events_df, var_id, var_id)

In [6]:
# Divide events into training, validation and testing
splits = save_restore_sex_fmri_splits(k=5)
ood_site_id = SITES[0]
events_train, events_id_test, events_ood_test = divide_events_by_splits(events_df, splits, ood_site_id)
print("Nr. events train: {}, val: {}, test: {}".format(len(events_train), len(events_id_test), len(events_ood_test)))

Nr. events train: 7064, val: 1738, test: 284


# Linear Regression

In [7]:
# Define PyTorch datasets and dataloaders
datasets = OrderedDict([('train', PandasDataset(events_train, feature_cols, target_col)),
            ('val', PandasDataset(events_id_test, feature_cols, target_col)),
            ('test', PandasDataset(events_ood_test, feature_cols, target_col))])

In [8]:
# Create dataloaders
batch_size = 64
dataloaders = OrderedDict([(dataset_name, DataLoader(dataset, batch_size=batch_size, shuffle=True))
    for dataset_name, dataset in datasets.items()])

for X, y in dataloaders['train']:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X: torch.Size([64, 177])
Shape of y: torch.Size([64]) torch.int64


In [9]:
# Determine device for training
device = "cpu" #("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [10]:
# Define model
models_path = os.path.join(output_path, 'ABCD_sex_prediction_lr', 'models')

model = LinearRegressor(save_path=models_path, input_size=len(feature_cols))

model = model.to(device)
print(model)

LinearRegressor(
  (linear): Linear(in_features=177, out_features=1, bias=True)
)


In [11]:
# Define optimizer and trainer
learning_rate = 1e-1
loss_f = nn.MSELoss()

trainer_path = os.path.join(output_path, 'ABCD_sex_prediction_lr', 'results')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

trainer = RegressorTrainer(trainer_path, device, optimizer, loss_f)

In [15]:
# Train model
trainer.train(model, dataloaders['train'], dataloaders, 
              nr_epochs=100, starting_from_epoch=0,
              print_loss_every=1, eval_every=10, export_every=50, verbose=True)

  0%|          | 0/100 [00:00<?, ?it/s]/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). Thi

Epoch 0
train MSELoss: 0.250 MSE: 0.249
val MSELoss: 0.250 MSE: 0.248
test MSELoss: 0.251 MSE: 0.250
Saved PyTorch model state LinearRegressor_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_prediction_lr/models
Saved trainer state RegressorTrainer_optimizer_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_prediction_lr/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_prediction_lr/results
Ending epoch 1, loss 0.2501215044711087
Ending epoch 2, loss 0.2500013605699883
Ending epoch 3, loss 0.2501028259326746
Ending epoch 4, loss 0.25005126603551814
Ending epoch 5, loss 0.2500514667313378


  8%|▊         | 8/100 [00:00<00:04, 22.95it/s]

Ending epoch 6, loss 0.2501188895186862
Ending epoch 7, loss 0.25014638350353585
Ending epoch 8, loss 0.250043273374841
Ending epoch 9, loss 0.24983439829435433
Ending epoch 10, loss 0.24997380362437652
Epoch 10
train MSELoss: 0.250 MSE: 0.248
val MSELoss: 0.250 MSE: 0.248
test MSELoss: 0.251 MSE: 0.250


/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results

...

/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results

Epoch 90
train MSELoss: 0.250 MSE: 0.248
val MSELoss: 0.250 MSE: 0.248
test MSELoss: 0.250 MSE: 0.249
Ending epoch 91, loss 0.24993389442160324
Ending epoch 92, loss 0.25005109473928677
Ending epoch 93, loss 0.24993374017444817
Ending epoch 94, loss 0.25018120456386256
Ending epoch 95, loss 0.2500508069186597


100%|██████████| 100/100 [00:04<00:00, 22.48it/s]

Ending epoch 96, loss 0.25012529151396706
Ending epoch 97, loss 0.2501210786469348
Ending epoch 98, loss 0.24994047657326535
Ending epoch 99, loss 0.2500727961997728
Ending epoch 100, loss 0.24996185343007785
Finished training
Epoch 100
train MSELoss: 0.250 MSE: 0.248
val MSELoss: 0.250 MSE: 0.248
test MSELoss: 0.252 MSE: 0.250
Saved PyTorch model state LinearRegressor_epoch100.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_prediction_lr/models
Saved trainer state RegressorTrainer_optimizer_epoch100.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_prediction_lr/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_prediction_lr/results



/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/carolinezanze/anaconda3/envs/cns/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect result